pareil que champ  crtl C + ctrl V go brrr

In [2]:
import requests
import polars as pl
import re

def get_latest_version():
    try:
        return requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    except:
        return "16.1.1"

def clean_html(raw_html):
    # Nettoyage des balises HTML pour une lecture propre dans le CSV
    if not raw_html: return ""
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    # On retire les sauts de ligne multiples pour que le CSV reste propre
    return cleantext.strip().replace('\n', ' ')

def get_base_shop_items():
    version = get_latest_version()
    print(f"🛒 Récupération du magasin (Patch {version})...")

    url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/item.json"
    response = requests.get(url)
    
    if response.status_code != 200:
        print("Erreur API")
        return None

    data = response.json()['data']
    items_list = []

    for item_id, details in data.items():
        # --- FILTRES STRICTS ---
        
        # 1. Uniquement Map 11 (Faille)
        if details.get('maps', {}).get('11') is not True:
            continue

        # 2. Uniquement les objets ACHETABLES (Vire les items Ornn, évolutions support, etc.)
        if not details.get('gold', {}).get('purchasable', False):
            continue

        # 3. Retire les objets réservés à un champion spécifique (Gangplank, Fiddle, Kalista)
        if 'requiredChampion' in details:
            continue
            
        # 4. (Optionnel) Retire les consommables gratuits bizarres ou trinkets "internes"
        # On garde si le prix total > 0 OU si c'est un ward/potion standard
        if details['gold']['total'] == 0 and "Trinket" not in details.get('tags', []):
             # Garde les Trinkets gratuits (Totem, Oracle), vire les trucs de dev
             pass 

        # --- CREATION DE LA LIGNE ---
        row = {
            "ID": item_id,
            "Name": details['name'],
            "Price": details['gold']['total'],
            "Sell_Price": details['gold']['sell'],
            "Description": clean_html(details.get('description', '')),
            "Short_Desc": clean_html(details.get('plaintext', '')),
            "ImageURL": f"https://ddragon.leagueoflegends.com/cdn/{version}/img/item/{details['image']['full']}"
        }
        items_list.append(row)

    return pl.DataFrame(items_list)

# --- EXÉCUTION ---
if __name__ == "__main__":
    df_shop = get_base_shop_items()
    
    if df_shop is not None:
        print(f"✅ Terminé ! {len(df_shop)} objets standards trouvés.")
        print(df_shop.head(5))
        
        # Sauvegarde
        # df_shop.write_csv("shop_items_base.csv", separator=";")

🛒 Récupération du magasin (Patch 16.1.1)...
✅ Terminé ! 250 objets standards trouvés.
shape: (5, 7)
┌──────┬─────────────────┬───────┬────────────┬─────────────────┬─────────────────┬────────────────┐
│ ID   ┆ Name            ┆ Price ┆ Sell_Price ┆ Description     ┆ Short_Desc      ┆ ImageURL       │
│ ---  ┆ ---             ┆ ---   ┆ ---        ┆ ---             ┆ ---             ┆ ---            │
│ str  ┆ str             ┆ i64   ┆ i64        ┆ str             ┆ str             ┆ str            │
╞══════╪═════════════════╪═══════╪════════════╪═════════════════╪═════════════════╪════════════════╡
│ 1001 ┆ Boots           ┆ 300   ┆ 210        ┆ 25  Move Speed  ┆ Slightly        ┆ https://ddrago │
│      ┆                 ┆       ┆            ┆                 ┆ increases Move  ┆ n.leagueoflege │
│      ┆                 ┆       ┆            ┆                 ┆ Speed           ┆ nd…            │
│ 1004 ┆ Faerie Charm    ┆ 200   ┆ 140        ┆ 50%  Base Mana  ┆ Slightly        ┆ https://